# Building Spark applications with Scala API

Apache Spark is written in Scala. Scala (along with Python and Java) is among three languages supported by Spark, and in fact Scala functionality is typically added the first to new Spark releases.


## Preparing our first Scala Spark application

Let us start with an PySpark application we have prepared on one of the previous steps. Here it is:

In [1]:
from pyspark import SparkContext
import sys
import time

def main1(args):
    start = time.time()
    #sc = SparkContext(appName="LoadUnstructured")

    input_rdd = sc.textFile("../2_LoadingData/data/unstructured/",10)
    counts = input_rdd.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

    print "\nTaking the 10 most frequent words in the text and corresponding frequencies:"
    print counts.takeOrdered(10, key=lambda x: -x[1])
    end = time.time()
    print "Elapsed time: ", (end-start)

In [2]:
main1(sys.argv)


Taking the 10 most frequent words in the text and corresponding frequencies:
[(u'the', 22635), (u'of', 11167), (u'and', 11086), (u'to', 10707), (u'a', 10433), (u'I', 10183), (u'in', 7006), (u'that', 6911), (u'was', 6779), (u'his', 4955)]
Elapsed time:  3.28145909309


In [ ]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.SparkContext._

object WordCount {
  def main(args: Array[String]) {

    val conf = new SparkConf().setAppName("WordCount")


    val textFile = spark.textFile("../loading_data/unstructured/",10)
    val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _)
    println("\nTaking the 10 most frequent words in the text and corresponding frequencies:")
    println(counts.takeOrdered(10).(Ordering[Int].reverse.on(x=>x._2)))
        
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0)/1000000000.)
    spark.stop()
  }
}

In [ ]:
#We will not run it from the notebook, because we are using Jupyter. 
#But we would need a dedicated for Scala applications, like Zepellin

Go back to the course working area on Adroit and change to the folder for this exercise:

```bash
cd 6_ScalaApps
```

## Submitting Scala Spark application Q/A

Q: So you've written some Spark code in Scala. How do you submit it to Spark and run it?  
A: Use `sbt` or `maven` to package it into a Java jar, and submit it to Spark using `spark-submit`

Q: What's a Java jar?  
A: JAR (Java Archive) is a package file format typically used to aggregate many Java class files and associated metadata and resources (text, images, etc.) into one file to distribute application software or libraries on the Java platform.

### Packaging with `sbt`

**What is SBT?**  
SBT is a modern build tool written in/for Scala, though it is also a general purpose build tool  

**Why SBT?**
- Good dependency management
- Full Scala language support for creating tasks
- Launch REPL in project context

Create a root directory for your project and run:
```bash
mkdir -p src/{main,test}/{resources,scala}
mkdir lib project
```
within it. 

This script will automatically create the proper `sbt` directory structure, which borrows from the Java `maven` directory structure. The script will also generate a template `build.sbt` file at the top of the directory that you should fill out with the appropriate versions and dependencies for your app.

Then we can take our Scala code, and put it in the src folder (you should have it in the main folder, so just move it there):

```bash
mv WordCount.scala src/main/scala/
```

**Project Layout (Directory structure)**   

`project` – project definition files  
`project/build/` *yourproject* `.scala` – the main project definition file  
`project/build.properties` – project, sbt and scala version definitions  
`src/main` – your app code goes here, in a subdirectory indicating the code’s language (e.g. src/main/scala, src/main/java)  
`src/main/resources` – static files you want added to your jar (e.g. logging config)  
`src/test` – like src/main, but for tests  
`lib_managed` – the jar files your project depends on. Populated by sbt update  
`target` – the destination for generated stuff (e.g. generated thrift
code, class files, jars)  

#### `build.sbt`: Dependencies and versioning

Example `simple.sbt` (located in the root directory of your project) 

```scala
name := "WordCount"

version := "1.0"

scalaVersion := "2.10.4"

libraryDependencies ++= Seq(
    // Spark dependency
    "org.apache.spark" % "spark-core_2.10" % "1.6.1" % "provided"
)
```


#### Assembly.sbt to build a fat Jar

Example assembly.sbt located in the /project folder of your project:

```scala
resolvers += Resolver.url("artifactory", url("http://scalasbt.artifactoryonline.com/scalasbt/sbt-plugin-releases"))(Resolver.ivyStylePatterns)

addSbtPlugin("com.eed3si9n" % "sbt-assembly" % "0.13.0")
```

### Running (submitting a `jar` to Spark)

1. Run `sbt assembly` in your project's home directory. The output to console will tell you the name and location of the resulting jar (under `./target`) 

You should now see the Jar file generated:
```bash
[alexeys@bd scala_spark] ll target/scala-2.10/
total 6968
drwxr-xr-x. 2 alexeys cses    4096 Dec  9 10:43 classes
-rw-r--r--. 1 alexeys cses 7129172 Dec  9 10:43 WordCount-assembly-1.0.jar
```

2. In the Slurm batch script, use spark-submit as usual to submit the Spark app, but you would need to specify the --class and the path to jar from the current folder, for instance:

```bash
spark-submit --class "WordCount" --total-executor-cores target/scala-2.10/WordCount-assembly-1.0.jar
```


### Hands-on mini-exercise

Run the above steps in your working area on Adroit.

# Model selection via cross validation

An important task in ML is model selection, or using data to find the best model or parameters for a given task. **Pipelines** facilitate model selection by making it easy to tune an entire **Pipeline** at once, rather than tuning each element in the **Pipelines** separately.

Currently, **spark.ml** supports model selection using the **CrossValidator** class, which takes an Estimator, a set of ParamMaps, and an Evaluator. CrossValidator begins by splitting the dataset into a set of folds which are used as separate training and test datasets; e.g., with k=3 there is 3 folds, CrossValidator will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. CrossValidator iterates through the set of ParamMaps. For each ParamMap, it trains the given Estimator and evaluates it using the given Evaluator.

The Evaluator can be a RegressionEvaluator for regression problems, a BinaryClassificationEvaluator for binary data, or a MultiClassClassificationEvaluator for multiclass problems. The default metric used to choose the best ParamMap can be overriden by the setMetric method in each of these evaluators.

The ParamMap which produces the best evaluation metric (averaged over the k folds) is selected as the best model. CrossValidator finally fits the Estimator using the best ParamMap and the entire dataset.

The following example demonstrates using CrossValidator to select from a grid of parameters. To help construct the parameter grid, we use the ParamGridBuilder utility.


In [ ]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.sql.Row

// Prepare training data from a list of (id, text, label) tuples.
val training = sqlContext.createDataFrame(Seq(
  (0L, "a b c d e spark", 1.0),
  (1L, "b d", 0.0),
  (2L, "spark f g h", 1.0),
  (3L, "hadoop mapreduce", 0.0),
  (4L, "b spark who", 1.0),
  (5L, "g d a y", 0.0),
  (6L, "spark fly", 1.0),
  (7L, "was mapreduce", 0.0),
  (8L, "e spark program", 1.0),
  (9L, "a e c l", 0.0),
  (10L, "spark compile", 1.0),
  (11L, "hadoop software", 0.0)
)).toDF("id", "text", "label")

// Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
val tokenizer = new Tokenizer()
  .setInputCol("text")
  .setOutputCol("words")
val hashingTF = new HashingTF()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("features")
val lr = new LogisticRegression()
  .setMaxIter(10)
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))

// We use a ParamGridBuilder to construct a grid of parameters to search over.
// With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
// this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
val paramGrid = new ParamGridBuilder()
  .addGrid(hashingTF.numFeatures, Array(10, 100, 1000))
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

// We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
// This will allow us to jointly choose parameters for all Pipeline stages.
// A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
// Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
// is areaUnderROC.
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(2) // Use 3+ in practice

// Run cross-validation, and choose the best set of parameters.
val cvModel = cv.fit(training)

// Prepare test documents, which are unlabeled (id, text) tuples.
val test = sqlContext.createDataFrame(Seq(
  (4L, "spark i j k"),
  (5L, "l m n"),
  (6L, "mapreduce spark"),
  (7L, "apache hadoop")
)).toDF("id", "text")

// Make predictions on test documents. cvModel uses the best model found (lrModel).
cvModel.transform(test)
  .select("id", "text", "probability", "prediction")
  .collect()
  .foreach { case Row(id: Long, text: String, prob: Vector, prediction: Double) =>
    println(s"($id, $text) --> prob=$prob, prediction=$prediction")
  }

### Hands-on mini-exercise

Run this on the Adroit cluster